In [ ]:
import subprocess
import sys,os

Instalasi library yang dibutuhkan dengan menggunakan requirements.txt

In [ ]:
subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-r', 'requirements.txt'], cwd=os.getcwd())

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
import os
import json

Path ke direktori dataset

In [ ]:
dataset_dir = 'dataset' 

Membuat ImageDataGenerator untuk memuat gambar dengan augmentasi data tambahan<br>
Jangan dirubah karna ini adalah yang paling bagus,kalau mau di rubah silakan research lagi

In [ ]:
datagen = ImageDataGenerator(
    validation_split=0.2, 
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

Generator untuk training data

In [ ]:
train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

Generator untuk validation data (tidak menggunakan augmentasi data)

In [ ]:
validation_datagen = ImageDataGenerator(validation_split=0.2, rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    dataset_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Membuat model yang lebih kompleks

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Melatih model

In [ ]:
history = model.fit(
    train_generator,
    epochs=20,  # Meningkatkan jumlah epoch untuk pelatihan yang lebih baik
    validation_data=validation_generator
)

Mengevaluasi model

In [ ]:
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

Menyimpan model

In [ ]:
model.save('model/image_classification_model.h5')

Menyimpan class indices

In [ ]:
class_indices = train_generator.class_indices
with open('model/class_indices.json', 'w') as f:
    json.dump(class_indices, f)